<a href="https://colab.research.google.com/github/GodJiLee/Dacon/blob/main/%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9%20%EC%8B%9D%EC%88%98%EC%9D%B8%EC%9B%90%20%EC%98%88%EC%B8%A1/96_514367651_LGBM_%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **To Do List**
1. 요일 등 변수 팩터화 (0~4) 
> https://teddylee777.github.io/scikit-learn/labelencoder-%EC%82%AC%EC%9A%A9%EB%B2%95   
> get_dummies, label_encoder, astype(‘category’).cat.codes
2. LGBMRegressor 성능 최적화 방법 찾기
> 1. GridSearch로 파라미터 최적화   
> 2. 논문 찾기   
> 3. xgb, lgbm 결합하기
3. 공유코드 1, 3과도 결합지점 찾기 (1. 앙상블)
> https://www.kaggle.com/janged/xgb-lgb
4. 시각화 자료보고 인사이트 얻기
5. random_state 바꿔보기
6. LGBMRegressor에 공유코드 2,4 결합 data 학습
7. 주/연도 정보도 feature에 추가하기 (O)

# **2. 한글 패치 (matplotlib)**

In [ ]:
# Step 1. Matplotlib 업그레이드
!pip install matplotlib -U

# Step 2. 한글 설치 및 사용 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# Step 3. 런타임 재시작

     |████████████████████████████████| 10.3 MB 5.3 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (6,981 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 160837 files and d

In [ ]:
# Step 4. 한글 글꼴 설정
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = ['NanumGothic', 'sans-serif'] # 차트의 폰트 설정
plt.rcParams['axes.unicode_minus'] = False

# **3. 기타 설정 (임포트 등)**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# 경고 제어
warnings.filterwarnings(action = 'ignore') # 경고를 출력하지 않음

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder # 수치형 자료가 아닌 자료에 대해 모델 학습을 위해 정수형으로 바꿔주는 기능
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor # Tree 기반 학습 알고리즘 (Gradient Boosting 프레임워크)

from gensim.models import Word2Vec
from gensim.models import KeyedVectors # 워드 벡터를 저장하고 관리하는 기능, 유사성 조회
from sklearn.manifold import TSNE # PCA 기반 차원 축소의 문제를 해결하기 위한 방법 (티스니)
from sklearn.decomposition import PCA

# **4. 데이터 불러오기**

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/test.csv')

# **5. 데이터 정제 (Dtype: object)**
1. 조식, 중식, 석식 메뉴는 버리기 (drop)
2. 일자, 요일은 라벨 인코딩 (숫자형으로 변환)

## **1. 조식, 중식, 석식 메뉴는 버리기 (drop)**

## **2. 일자, 요일은 라벨 인코딩 (숫자형으로 변환)**

In [ ]:
# 월, 일 인코딩
train['연'] = pd.DatetimeIndex(train['일자']).year
test['연'] = pd.DatetimeIndex(test['일자']).year

train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month

train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week

train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

In [ ]:
# 요일 인코딩
# 팩터화 해야 하는거 아니야...?
weekday = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5
}

train['요일'] = train['요일'].map(weekday)
test['요일'] = test['요일'].map(weekday)

# **6. 데이터 정제 (아이디어)**
* 휴가자, 재택근무자는 중식/석식에 참여하지 않을 것이므로 본사 정원수에서 차감한다.
* 즉, 본사정원수 - 휴가자 - 재택근무자
* 식사참여율 = 식사계 / 식사가능자수

In [ ]:
train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

In [ ]:
# 식사가능자수 float -> int로 변경
train['식사가능자수'] = train['식사가능자수'].astype('int')
test['식사가능자수'] = test['식사가능자수'].astype('int')

In [ ]:
train['중식참여율'] = train['중식계'] / train['식사가능자수']
train['석식참여율'] = train['석식계'] / train['식사가능자수']

정제한 데이터를 정리하면,
### **요일(석식) 컬럼 만들기**

In [ ]:
# 요일을 석식 rank에 맞춰 mapping한 요일(석식) 컬럼 만들기
# train 데이터만의 특징일 수 있지 않을까..?
weekday_rank4dinner = {
    1: 1,
    2: 2,
    3: 5,
    4: 3,
    5: 4,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

### **Label Encoder로 범주형 자료 수치형으로 바꾸기**

In [ ]:
# 수치형으로 바꾸지 않은 상태에서 encoder 적용..?
cat_features = ['요일', '요일(석식)']

In [ ]:
for feature in cat_features:
    le = LabelEncoder()
    le.fit(train[feature])
    train[feature] = le.transform(train[feature])
    test[feature] = le.transform(test[feature])

In [ ]:
# lunch

# train
x_lunch = train[['연', '월', '주', '일', '요일', '식사가능자수', '본사출장자수', '본사시간외근무명령서승인건수']]
y_lunch = train['중식계']
# test
test_x_lunch = test[['연', '월', '주', '일', '요일', '식사가능자수', '본사출장자수', '본사시간외근무명령서승인건수']]

In [ ]:
# dinner

# train
x_dinner = train[['연', '월', '주', '일', '요일(석식)', '식사가능자수', '본사출장자수', '본사시간외근무명령서승인건수']]
y_dinner = train['석식계']
# test
test_x_dinner = test[['연', '월', '주', '일', '요일(석식)', '식사가능자수', '본사출장자수', '본사시간외근무명령서승인건수']]

# **Modeling**

### **LGBMRegressor**

In [ ]:
# Simple LGBM Regressor w/o tuning
model_lunch = LGBMRegressor()
model_lunch.fit(x_lunch, y_lunch)

model_dinner = LGBMRegressor()
model_dinner.fit(x_dinner, y_dinner)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
# Prediction
pred_lunch = model_lunch.predict(test_x_lunch)
pred_dinner = model_dinner.predict(test_x_dinner)

# **Inference**

In [ ]:
# Inference
test_pred_lunch = model_lunch.predict(test_x_lunch)
test_pred_dinner = model_dinner.predict(test_x_dinner)

submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/sample_submission.csv')

In [ ]:
submission_df.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


In [ ]:
submission_df['중식계'] = test_pred_lunch
submission_df['석식계'] = test_pred_dinner

In [ ]:
# Save

submission_df.to_csv('sub_2_연도주포함_공유코드결합.csv', index = False)